In [ ]:
!pip -q install langchain
!pip -q install bitsandbytes accelerate
!pip -q install datasets loralib sentencepiece
!pip -q install pypdf
!pip -q install sentence_transformers
!pip -q install openai
!pip -q install tiktoken
!pip -q install unstructured
!pip install tokenizers
!pip install faiss-cpu
!pip install xformers
!pip install pinecone-client
!pip install transformers==4.30.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... 

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS, Pinecone
import pinecone
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import HuggingFacePipeline

from transformers import AutoTokenizer , AutoModelForCausalLM,pipeline
from huggingface_hub import notebook_login
import sys
import textwrap
import torch
import os
import nltk
nltk.download('puntk')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading puntk: Package 'puntk' not found in index
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
os.environ['OPENAI_API_KEY']=""

In [ ]:
URL=['https://www.unitedhealthgroup.com',
     'https://careers.unitedhealthgroup.com/job-search-results/?keyword=data%20scientist']
loader=UnstructuredURLLoader(urls=URL)
data=loader.load()
data

[Document(page_content='Skip to content\n\nOur People & Businesses\n                \n              \n              \n                \n                  \n                    Our People & Businesses \n                    We are a health care and well-being company made up of a diverse team around the world dedicated to making health care work better through two distinct and complementary businesses: Optum and UnitedHealthcare.\n\t\t\t\t\t\n                    \n                        \n                  \n\t\t\t\t\t\n\t\t\t\t\t\t\n                    \t\tMission & Values \n                                \n                        \t\t                        \n                    \t\t\n\t\t\t\t\t\t\n                    \t\tOur People & Culture\n                                \n                        \t\t\n                          \t\t\tJoin Our Team\n                                        \n                                    \n                        \t\t\n                       

In [ ]:
len(data)

2

In [ ]:
text_splitter=CharacterTextSplitter(separator='\n',chunk_size=1000,chunk_overlap=200)
text_chunk=text_splitter.split_documents(data)

In [ ]:
len(text_chunk)

16

In [ ]:
text_chunk[0]

Document(page_content='Skip to content\nOur People & Businesses\n                \n              \n              \n                \n                  \n                    Our People & Businesses \n                    We are a health care and well-being company made up of a diverse team around the world dedicated to making health care work better through two distinct and complementary businesses: Optum and UnitedHealthcare.\n\t\t\t\t\t\n                    \n                        \n                  \n\t\t\t\t\t\n\t\t\t\t\t\t\n                    \t\tMission & Values \n                                \n                        \t\t                        \n                    \t\t\n\t\t\t\t\t\t\n                    \t\tOur People & Culture\n                                \n                        \t\t\n                          \t\t\tJoin Our Team\n                                        \n                                    \n                        \t\t\n                          

In [ ]:
embeddings=HuggingFaceEmbeddings()

In [ ]:
query_result=embeddings.embed_query("Hello World")
len(query_result)

768

In [ ]:
vector_stores=FAISS.from_documents(text_chunk,embeddings)
vector_stores

In [ ]:
llm=ChatOpenAI()

In [ ]:
notebook_login()

In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM,BitsAndBytesConfig

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              load_in_8bit=True,
                                              #load_in_4bit=True
                                             )

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:
llm=HuggingFacePipeline(pipeline=pipe,model_kwargs={'temperature': 0})

In [ ]:
llm.predict("Please Tell me about Edgeforce Solution Private Limited")

"?\n\nEdgeforce Solution Private Limited is a leading IT consulting and software development company based in India. The company was established in 2007 and has since then been providing various services to clients across the globe. Edgeforce Solution Private Limited offers a wide range of services, including web development, mobile app development, e-commerce solutions, digital marketing, and more.\nWith a team of highly skilled professionals, Edgeforce Solution Private Limited has successfully delivered numerous projects for clients in various industries. Their clientele includes small and medium-sized businesses as well as large corporations, and they have a proven track record of delivering high-quality solutions within the agreed timelines.\nEdgeforce Solution Private Limited is known for its unique and innovative approach to software development, which sets them apart from their competitors. The company believes in understanding the clients' business needs and tailoring their sol

In [ ]:
chain=RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=vector_stores.as_retriever())

In [ ]:
result=chain({'question':''},return_only_outputs=True)

In [ ]:
result['answer']

' CBM stands for Condition-Based Maintenance.\n\n\n\n\n\n\n\n'

In [ ]:
wrapped_text=textwrap.fill(result['answer'],width=50)
wrapped_text

' Edgedorce is a company that specializes in data\nscience and engineering, and has received\nrecognition and funding for their work in this\nfield.'

In [ ]:
while True:
  query=input(f"Prompt: ")
  if query== 'exit':
    print("Shutting Down")
    sys.exit()
  if query == '':
    continue
  result=chain({'question':query})
  print(f"Answer:" +result['answer'])

Prompt: Tell me something about UnitedHealth Group
Answer: UnitedHealth Group is a healthcare and well-being company that operates through two businesses: Optum and UnitedHealthcare. They offer a range of healthcare services and products, and have reported their second quarter 2023 financial results.


Prompt: Are there any open Positions?
Answer: Yes, there are currently open positions available within the Disability Inclusion Programs as listed on the search results page. 






Prompt: ok then list down those open position please


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Answer: I apologize, but none of the provided sources mention any open positions at Optum or UnitedHealth Group.




Prompt: Ok are there any open position for data scientist if yes the please list down those
Answer: Yes, there are open positions for data scientists at Optum and UnitedHealth Group.





















Prompt: exit
Shutting Down


SystemExit: ignored

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
